# Classificador de Sentiments a Xarxes Socials en Català (CSXSC): Fine-Tuning Model

**Author:** Daniel Arias Cámara  
**Date:** 07-2025  

**Description:**  This notebook presents the fine-tuning process of the **CSXSC** model, designed to classify social media reviews in Catalan into three categories: positive, negative, and neutral.

The base model for CSXSC is [roberta-base-ca-v2](https://huggingface.co/projecte-aina/roberta-base-ca-v2), a Catalan adaptation of RoBERTa, an encoder-only transformer architecture well-suited for text classification tasks.

This model is provided by Aina Kit and trained on a diverse Catalan corpus, including Catalan Crawling, Wikipedia, the Official Gazette of the Government of Catalonia (DOGC), and other publicly available datasets.

In [1]:
from datasets import load_dataset

data_files = {
    "train": "./train.csv",
    "validation": "./validation.csv",
    "test": "./test.csv"
}

dataset = load_dataset("csv", data_files=data_files)

/home/user/anaconda3/envs/ai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 19192 examples [00:00, 380989.96 examples/s]
Generating validation split: 2399 examples [00:00, 298845.72 examples/s]
Generating test split: 2400 examples [00:00, 266636.55 examples/s]


In [2]:
print(dataset["train"][0])

{'text': 'Odio la poesia', 'label': 'negative'}


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
tokenizer.padding_side = "right"

model = AutoModelForSequenceClassification.from_pretrained("projecte-aina/roberta-base-ca-v2", num_labels=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at projecte-aina/roberta-base-ca-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
label2id = {"negative": 0, "neutral": 1, "positive": 2}

def map_label(example):
    if isinstance(example["label"], str):
        label_lower = example["label"].strip().lower()
        if label_lower in label2id:
            example["label"] = label2id[label_lower]
        else:
            raise ValueError(f"Unexpected label: {example['label']}")
    return example

dataset = dataset.map(map_label)

Map: 100%|██████████| 2400/2400 [00:00<00:00, 50762.62 examples/s]


In [5]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 2400/2400 [00:00<00:00, 24759.22 examples/s]


In [6]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

In [7]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "eval_accuracy": accuracy_score(labels, predictions),
        "eval_f1_macro": f1_score(labels, predictions, average="macro"),
        "eval_f1_weighted": f1_score(labels, predictions, average="weighted"),
    }

# Hyperparameters
learning_rates = [2e-5]
batch_sizes = [16]
weight_decays = [0.01]
epoch_options = [4]

best_score = -float("inf")
best_params = None
training_history = []

for lr in learning_rates:
    for batch_size in batch_sizes:
        for wd in weight_decays:
            for epochs in epoch_options:
                print(f"\nTraining with LR={lr}, BS={batch_size}, WD={wd}, Epochs={epochs}")

                training_args = TrainingArguments(
                    output_dir="./results",
                    eval_strategy="epoch",
                    save_strategy="no",
                    learning_rate=lr,
                    per_device_train_batch_size=batch_size,
                    per_device_eval_batch_size=batch_size,
                    num_train_epochs=epochs,
                    weight_decay=wd,
                    logging_strategy="epoch",
                    report_to="none"
                )

                trainer = Trainer(
                    model=model,
                    args=training_args,
                    train_dataset=tokenized_datasets["train"],
                    eval_dataset=tokenized_datasets["validation"],
                    tokenizer=tokenizer,
                    compute_metrics=compute_metrics
                )

                train_result = trainer.train()
                eval_metrics = trainer.evaluate()

                eval_metrics.update({
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "weight_decay": wd,
                    "epochs": epochs,
                    "train_loss": train_result.training_loss
                })
                training_history.append(eval_metrics)

                if eval_metrics["eval_f1_macro"] > best_score:
                    best_score = eval_metrics["eval_f1_macro"]
                    best_params = (lr, batch_size, wd, epochs)

print(f"\nBest params: LR={best_params[0]}, BS={best_params[1]}, WD={best_params[2]}, "
      f"Epochs={best_params[3]} (F1-macro={best_score:.4f})")


/tmp/ipykernel_36354/770503976.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Training with LR=2e-05, BS=16, WD=0.01, Epochs=4


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,0.513800,0.429695,0.819925,0.801435,0.816418
2,0.370800,0.398182,0.836599,0.819002,0.834303
3,0.287800,0.431470,0.838683,0.822798,0.837652
4,0.229800,0.487156,0.838266,0.822186,0.837109



Best params: LR=2e-05, BS=16, WD=0.01, Epochs=4 (F1-macro=0.8222)
